In [ ]:
# Step 1: Upload your dataset
from google.colab import files
uploaded = files.upload()  # Select logistics_dataset_1000.csv from your computer



Saving logistics_dataset_1000.csv to logistics_dataset_1000.csv


In [ ]:
# Step 2: Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import folium

In [ ]:
# Step 3: Load dataset
df = pd.read_csv("logistics_dataset_1000.csv")
print("Dataset loaded successfully!")
print(df.head())

Dataset loaded successfully!
   package_id  origin_lat  origin_lon  destination_lat  destination_lon  \
0           1   12.956821   77.588053        12.967460        77.583223   
1           2   12.987833   77.569639        12.959984        77.567253   
2           3   12.997179   77.584276        12.962576        77.550644   
3           4   12.996065   77.595392        12.974234        77.567506   
4           5   12.967103   77.551813        12.989572        77.573205   

   distance_km  traffic_level  package_weight_kg vehicle_type time_of_day  \
0        14.93              3                7.5          Van   Afternoon   
1         2.32              3                1.8         Bike     Evening   
2         7.86              3                2.6         Bike   Afternoon   
3        14.82              3                6.1         Bike     Morning   
4         9.74              3                5.8        Truck     Morning   

  weather  delivery_time_min  
0   Rainy              68.

In [ ]:
# Step 4: Encode categorical features
le_vehicle = LabelEncoder()
le_time = LabelEncoder()
le_weather = LabelEncoder()

df['vehicle_type_enc'] = le_vehicle.fit_transform(df['vehicle_type'])
df['time_of_day_enc'] = le_time.fit_transform(df['time_of_day'])
df['weather_enc'] = le_weather.fit_transform(df['weather'])

In [ ]:
# Step 5: Prepare features and target
X = df[['distance_km', 'traffic_level', 'package_weight_kg',
        'vehicle_type_enc', 'time_of_day_enc', 'weather_enc']]
y = df['delivery_time_min']

In [ ]:
# Step 6: Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Step 7: Scale features (optional but recommended)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Step 8: Train Linear Regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [ ]:
# Step 9: Evaluate model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
print(f"R2 Score: {r2:.2f}")

Mean Squared Error: 111.53
R2 Score: 0.84


In [ ]:
# Step 10: Predict delivery time for a sample package
sample = pd.DataFrame({
    'distance_km':[10],
    'traffic_level':[2],
    'package_weight_kg':[5],
    'vehicle_type_enc':[le_vehicle.transform(['Van'])[0]],
    'time_of_day_enc':[le_time.transform(['Morning'])[0]],
    'weather_enc':[le_weather.transform(['Sunny'])[0]]
})
sample_scaled = scaler.transform(sample)
pred_time = model.predict(sample_scaled)
print("Predicted Delivery Time (min):", round(pred_time[0], 2))


Predicted Delivery Time (min): 30.96


In [ ]:
# Step 11: Optional - Map Visualization of Origin & Destination
# Example coordinates (replace with real origin/destination if desired)
origin_lat, origin_lon = 12.9716, 77.5946
destination_lat, destination_lon = 12.9750, 77.6050

map_demo = folium.Map(location=[12.9733, 77.5998], zoom_start=13)
folium.Marker([origin_lat, origin_lon], tooltip="Origin", icon=folium.Icon(color='green')).add_to(map_demo)
folium.Marker([destination_lat, destination_lon], tooltip="Destination", icon=folium.Icon(color='red')).add_to(map_demo)
folium.PolyLine([[origin_lat, origin_lon], [destination_lat, destination_lon]], color="blue", weight=2.5, opacity=1).add_to(map_demo)
map_demo

In [ ]:
!pip install gradio


In [ ]:
import folium

def create_map(origin_lat, origin_lon, dest_lat, dest_lon):
    # Create base map
    m = folium.Map(location=[origin_lat, origin_lon], zoom_start=13)

    # Add markers
    folium.Marker([origin_lat, origin_lon], popup="Origin", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker([dest_lat, dest_lon], popup="Destination", icon=folium.Icon(color="red")).add_to(m)

    # Add route line
    folium.PolyLine([[origin_lat, origin_lon], [dest_lat, dest_lon]],
                    color="blue", weight=2.5, opacity=1).add_to(m)

    return m._repr_html_()


In [ ]:
def predict_and_map(distance, traffic, weight, vehicle, time_of_day, weather):
    try:
        # Normalize inputs to match LabelEncoder
        vehicle = vehicle.capitalize()
        time_of_day = time_of_day.capitalize()
        weather = weather.capitalize()

        # Safety check: if input not in encoder classes
        if vehicle not in le_vehicle.classes_:
            return f"Error: Vehicle '{vehicle}' not in {list(le_vehicle.classes_)}", ""
        if time_of_day not in le_time.classes_:
            return f"Error: Time '{time_of_day}' not in {list(le_time.classes_)}", ""
        if weather not in le_weather.classes_:
            return f"Error: Weather '{weather}' not in {list(le_weather.classes_)}", ""

        # Encode features
        vehicle_enc = le_vehicle.transform([vehicle])[0]
        time_enc = le_time.transform([time_of_day])[0]
        weather_enc = le_weather.transform([weather])[0]

        # Create DataFrame (preserve feature names)
        sample_df = pd.DataFrame([[distance, traffic, weight, vehicle_enc, time_enc, weather_enc]],
                                 columns=['distance_km','traffic_level','package_weight_kg',
                                          'vehicle_type_enc','time_of_day_enc','weather_enc'])

        # Scale & predict
        sample_scaled = scaler.transform(sample_df)
        pred_time = model.predict(sample_scaled)[0]

        # Map
        origin_lat, origin_lon = 12.9716, 77.5946
        dest_lat, dest_lon = 12.9750 + distance*0.001, 77.6050 + distance*0.001
        map_html = create_map(origin_lat, origin_lon, dest_lat, dest_lon)

        return f"Predicted Delivery Time: {pred_time:.2f} minutes", map_html

    except Exception as e:
        return f"Error during prediction: {str(e)}", ""





In [ ]:
# Step 12: Gradio interface
import gradio as gr
iface = gr.Interface(
    fn=predict_and_map,
    inputs=[
        gr.Number(label="Distance (km)"),
        gr.Slider(1,3,1,label="Traffic Level"),
        gr.Number(label="Package Weight (kg)"),
        gr.Dropdown(["Bike","Van","Truck"], label="Vehicle"),
        gr.Dropdown(["Morning","Afternoon","Evening"], label="Time of Day"),
        gr.Dropdown(["Sunny","Rainy","Cloudy"], label="Weather")
    ],
    outputs=[gr.Textbox(label="Prediction"), gr.HTML(label="Route Map")]
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e42ca5136a0631974.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
